In [5]:
import torch
import numpy as np
import math
from scipy.linalg import null_space
from numpy.linalg import svd
from scipy.optimize import linprog
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.nn as nn
from torch.autograd.variable import Variable
from torchvision import datasets, models, transforms

In [6]:
def nullspace(A, atol=1e-13, rtol=0):
    A = np.atleast_2d(A)
    u, s, vh = svd(A)
    tol = max(atol, rtol * s[0])
    nnz = (s >= tol).sum()
    ns = vh[nnz:].conj().T
    return ns

def Pc(z):
    z[z < 0] = 0
    return z

In [7]:
class admm:
    def __init__(self, c, A, b, rho, alpha):
        self.c = c
        self.A = A
        self.b = b
        self.rho = torch.tensor(rho, dtype=torch.double)
        self.alpha = torch.tensor(alpha, dtype=torch.double)
        val = np.linalg.lstsq(self.A.numpy(), self.b.numpy())
        self.x0 = torch.from_numpy(val[0])
        self.Q = torch.from_numpy(nullspace(self.A))
        self.omega = torch.zeros(self.Q.shape[1], 1, dtype=torch.double)
        self.z = self.x0 + torch.matmul(self.Q, self.omega)
        self.mu = torch.zeros(self.x0.shape, dtype=torch.double)
        
    def iterate(self):
        val1 = torch.matmul(self.Q.t(), self.Q).inverse()
        val2 = torch.matmul(self.Q.t(), self.z - self.x0 - self.mu)
        val3 = 1/self.rho * torch.matmul(self.Q.t(), self.c)
        
        self.omega = torch.matmul(val1, val2 - val3)
        
        self.z = Pc(torch.matmul(self.Q, self.omega) + self.x0 + self.mu)
        
        self.mu = self.mu + torch.matmul(self.Q, self.omega) - self.z + self.x0

In [8]:
def gaussian(ins, mean = 5, stddev=1):
    noise = Variable(ins.data.new(ins.size()).normal_(mean, stddev))
    if(torch.min(ins + noise) < 0):
        return gaussian(ins, mean, stddev)
    else:
        return ins + noise

In [9]:
def getAnswer(c, A, b):
    return linprog(c.view(c.shape[0]), A_eq = A, b_eq = b)

In [10]:
M = 10
N = 20
A = torch.rand(M, N, dtype=torch.double) * 10
c = torch.rand(N, 1, dtype=torch.double) * 10
samplex = torch.rand(N, 1, dtype=torch.double)
b = torch.matmul(A, samplex)

In [12]:
opt = admm(c,A,b,1.5,1.5)
print(opt.mu.shape)

torch.Size([20, 1])


/anaconda2/envs/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:8: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


In [19]:
import torch
from torch import nn
from torch import optim
import numpy as np

In [20]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(20, 1)
    def forward(self, x):
        return self.fc1(x)


In [22]:
net = Net()
# create a stochastic gradient descent optimizer
optimizer = optim.SGD(net.parameters(), lr=.5, momentum=0.9)
# create a loss function
criterion = nn.NLLLoss()

In [27]:
data 

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], dtype=torch.float64)

In [34]:
for i in range(100):
    opt = admm(gaussian(c),gaussian(A),gaussian(b),1,1)
    for i in range(20):
        data = Variable(opt.mu.t())
        target = Variable(torch.zeros(1,1))
        optimizer.zero_grad()
        print(data)
        net_out = net(data)
        loss = criterion(net_out, target)
        loss.backward()
        optimizer.step()

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       dtype=torch.float64)


/anaconda2/envs/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:8: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #4 'mat1'